In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import plotly.express as px

In [2]:
def pre_csv(df):
    df.dtm = pd.to_datetime(df.dtm)
    return df

def pre_dwd(df):
    df = df.to_dataframe().reset_index().rename(columns={"ref_datetime":"reference_time", "valid_datetime":"valid_time"})
    df.reference_time = df.reference_time.dt.tz_localize("UTC")
    df.valid_time = df.reference_time + df.valid_time * pd.Timedelta(1, "h")
    return df

def pre_ncep(df):
    return df

def pinball_score(y, q, alpha):
    return (y-q)*alpha*(y>=q) + (q-y)*(1-alpha)*(y<q)

Daten einlesen

In [3]:
df_pes_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc"))
df_pes_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20231027_20240108.nc"))
df_pes_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20240108_20240129.nc"))
df_pes_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20240129_20240519.nc"))
# 9831960 rows
df_pes = pd.concat([df_pes_0, df_pes_1, df_pes_2, df_pes_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_pes_0, df_pes_1, df_pes_2, df_pes_3
# 491319 rows
df_pes = df_pes.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["point", "longitude", "latitude"])
# 1273203 rows
df_pes = df_pes.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()


df_hornsea_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc"))
df_hornsea_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20231027_20240108.nc"))
df_hornsea_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240108_20240129.nc"))
df_hornsea_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240129_20240519.nc"))
# 17697528 rows
df_hornsea = pd.concat([df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3
# 491319 rows
df_hornsea = df_hornsea.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["longitude", "latitude"])
# 1273203 rows
df_hornsea = df_hornsea.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()


df_0 = pre_csv(pd.read_csv("data/Energy_Data_20200920_20240118.csv"))
df_1 = pre_csv(pd.read_csv("data/Energy_Data_20240119_20240519.csv"))
# 64224 rows
df = pd.concat([df_0, df_1]).sort_values(["dtm"]).reset_index(drop=True)
del df_0, df_1

df["Wind_MWh_credit"] = 0.5*df["Wind_MW"] - df["boa_MWh"] # Umrechnen in MWh und Abzug von BOA (BOA ist die Drosselung, die schon in MW miteingerechnet ist. Es wird aber mehr Strom produziert und vergütet, dashalb muss BOA wieder draufgerechnet werde, BOA ist immer negativ)
df["Solar_MWh_credit"] = 0.5*df["Solar_MW"]
df["Total_MWh_credit"] = df.Wind_MWh_credit + df.Solar_MWh_credit
# Sicherstellen, dass es immer 30min Schritte sind?

Daten zusammenführen

In [4]:
# Prüfen ob hierbei etwas verloren geht!!
# 1273203 rows
df_full = pd.merge(df_pes, df_hornsea, on=["reference_time", "valid_time"])
# 1273203 rows
df_full = df_full.merge(df[["dtm", "Wind_MWh_credit", "Solar_MWh_credit", "Total_MWh_credit"]], left_on="valid_time", right_on="dtm", how="left")

Feature Engineering

In [5]:
df_full["forcast_hours"] = (df_full.valid_time - df_full.reference_time) / pd.Timedelta(1, "h")
df_full["year"] = df_full.valid_time.dt.year
df_full["month"] = df_full.valid_time.dt.month
df_full["day"] = df_full.valid_time.dt.day
df_full["hour"] = df_full.valid_time.dt.hour

In [6]:
df_full.head()

,reference_time,valid_time,CloudCover,SolarDownwardRadiation,Temperature_x,RelativeHumidity,Temperature_y,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,dtm,Wind_MWh_credit,Solar_MWh_credit,Total_MWh_credit,forcast_hours,year,month,day,hour
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,0.450405,0.0,13.646173,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604,2020-09-20 00:00:00+00:00,498.142,0.0,498.142,0.0,2020,9,20,0
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,0.472211,0.0,13.658508,85.012253,15.412510,61.203667,61.726974,9.905537,11.648819,2020-09-20 00:30:00+00:00,478.788,0.0,478.788,0.5,2020,9,20,0
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,0.494018,0.0,13.670843,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033,2020-09-20 01:00:00+00:00,470.522,0.0,470.522,1.0,2020,9,20,1
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,0.520214,0.0,13.732101,84.357880,15.451218,60.511028,61.111038,9.631039,11.354128,2020-09-20 01:30:00+00:00,482.183,0.0,482.183,1.5,2020,9,20,1
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,0.546410,0.0,13.793360,83.904999,15.494086,60.202801,60.853306,9.494630,11.213223,2020-09-20 02:00:00+00:00,459.216,0.0,459.216,2.0,2020,9,20,2


Vorverarbeitung

In [7]:
# Hier könnte man die Daten beispielsweise skalieren oder ähnliches...
# Man kann die downtimes auf die labels draufrechenn, das könnte eine verbesserung sein

Testdaten abspalten

In [8]:
df_train = df_full.loc[df_full.reference_time < "2023-05-20"]
df_test = df_full.loc[df_full.reference_time >= "2023-05-20"]
del df_full

Trainingsset erstellen

In [11]:
label = "Wind_MWh_credit"
# label = "Solar_MWh_credit"
# label = "to_MWh_credit"
columns = ["forcast_hours", "year", "month", "day", "hour", 'CloudCover', 'SolarDownwardRadiation', 'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100']

index = df_train[df_train[label].isna()].index
x = df_train.drop(index)[columns].to_numpy()
y = df_train.drop(index)[label].to_numpy()

index_test = df_test[df_test[label].isna()].index
x_test = df_test.drop(index_test)[columns].to_numpy()
y_test = df_test.drop(index_test)[label].to_numpy()

Modell

In [10]:
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor(device="cuda", max_depth=2, n_estimators=1000, objective="reg:quantileerror", quantile_alpha=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
model.fit(x, y)

In [21]:
preds_train = model.predict(x)
preds = model.predict(x_test)

print(f"Pinballscore train = {np.array([pinball_score(y, pred, (i+1)/10).mean() for i, pred in enumerate(preds_train.T)]).mean()}")
print(f"Pinballscore test = {np.array([pinball_score(y_test, pred, (i+1)/10).mean() for i, pred in enumerate(preds.T)]).mean()}")

c:\Daten\Studieren\HKA\venv\DoPro2\Lib\site-packages\xgboost\core.py:158: UserWarning: [10:13:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Pinballscore train = 26.463347585572514
Pinballscore test = 48.67356901954439


In [39]:
# model.save_model("model_quantiles.json")
# model = XGBRegressor()
# model.load_model("model_quantiles.json")

In [41]:
# for i, pred in enumerate(preds.T):
#     print(f"Pinball-Score {10*(i+1)}%-Quantil: {pinball_score(y_test, pred, (i+1)/10).mean()}")

# model.get_booster().get_score()


In [42]:
# model = XGBRegressor(max_depth=6, n_estimators=500)#, objective="reg:quantileerror"
# grid = {
#     "max_depth": [4, 6, 8],
#     "n_estimators": [100, 300, 500],
#     "learning_rate": [0.1]
# }
# search = GridSearchCV(model, grid, cv=4).fit(x_train, y_train)
# search.best_params_